In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import copy
from sklearn.linear_model import Ridge
import requests

In [2]:
def readQAP_from_URL(url):
    """
    Lee un archivo DAT desde una URL y lo transforma en un diccionario que contiene el número de nodos (n), matriz de flujo (f) 
    y matriz de distancia (d)

    Args:
        url (str): URL del archivo .dat a leer

    Returns:
        instance_dict (dict): Diccionario que contiene el número de nodos (n), matriz de flujo (f) y matriz de distancia (d)
    """
    response = requests.get(url)
    content = response.text.split('\n')
    
    n_nodos = int(content[0])
    flow_matrix = np.array([list(map(int, line.split())) for line in content[1:n_nodos+1]])
    distance_matrix = np.array([list(map(int, line.split())) for line in content[n_nodos+2:2*n_nodos+2]])
    
    instance_dict = {'n': n_nodos, 'f': flow_matrix, 'd': distance_matrix}
    
    return instance_dict


def evaluateQAP(sol, f, d):
    """
    Evaluar QAP con una solución dada

    Args:
        sol (numpy array): vector con valores de posición para evaluar la función
        f (numpy array): matriz de flujo entre nodos
        d (numpy array): matriz de distancia entre nodos

    Returns:
        Valor de fitness del QAP en base a la solución entregada
    """
    
    fitness = np.sum(f * d[sol[:, None], sol])
    
    return fitness


def generate_population(size, n):
    return [np.random.permutation(n) for _ in range(size)]

def fitness(solution, instance):
    return evaluateQAP(solution, instance['f'], instance['d'])


def crossover(parent1, parent2):
    # Perform ordered crossover
    size = len(parent1)
    idx1, idx2 = sorted(np.random.choice(range(size), 2, replace=False))
    child = -np.ones(size, dtype=int)
    child[idx1:idx2] = parent1[idx1:idx2]
    mask = np.isin(parent2, child)
    child[child==-1] = parent2[~mask]
    return child

def selection(population, scores, k=3):
    # Select k random individuals from the population and return the best one
    selection_idx = np.random.choice(len(population), k, replace=False)
    return population[min(selection_idx, key=lambda idx: scores[idx])]

def mutation(solution):
    # Perform swap mutation
    idx1, idx2 = np.random.choice(range(len(solution)), 2, replace=False)
    solution[idx1], solution[idx2] = solution[idx2], solution[idx1]
    return solution

def replace_with_new_population(population, new_population, scores):
    # Replace the old population with the new population
    population_size = len(population)
    new_scores = [fitness(solution, instance) for solution in new_population]
    population = population + new_population
    scores = scores + new_scores
    idx = np.argsort(scores)[:population_size]
    return [population[i] for i in idx], [scores[i] for i in idx]


def population_diversity(population):
    # A simple measure of population diversity is the standard deviation of the solutions
    return np.std([np.sum(sol) for sol in population])


def genetic_algorithm(instance, n_population, n_iterations, mutation_rate):
    population = generate_population(n_population, instance['n'])
    scores = [fitness(solution, instance) for solution in population]

    for _ in range(n_iterations):
        
        print("Iteración ", _)
        
        new_population = []
        for _ in range(n_population):
            parent1 = selection(population, scores)
            parent2 = selection(population, scores)
            child = crossover(parent1, parent2)
            if np.random.rand() < mutation_rate:
                child = mutation(child)
            new_population.append(child)
        population, scores = replace_with_new_population(population, new_population, scores)
        
        print("Menor Costo:", min(scores))
        
    
    best_solution = population[np.argmin(scores)]
    best_score = min(scores)
    
    
    
    return best_solution, best_score


In [11]:
class LinUCB:
    def __init__(self, alpha=1, lambda_=1, min_value=0, max_value=1):
        self.alpha = alpha
        self.lambda_ = lambda_
        self.min_value = min_value
        self.max_value = max_value
        self.A = self.lambda_ 
        self.b = 0

    def update(self, reward, context):
        context = (context - self.min_value) / (self.max_value - self.min_value) # Normalización
        self.A += context * context
        self.b += reward * context

    def predict(self):
        context = 1 # Dado que ya no tenemos un contexto para predecir, simplemente usamos 1
        theta = self.b / self.A
        p = context * theta + self.alpha * np.sqrt(context / self.A)
        return p * (self.max_value - self.min_value) + self.min_value # Desnormalización


In [6]:
instance = readQAP_from_URL("https://www.opt.math.tugraz.at/qaplib/data.d/tho40.dat")

In [13]:
def genetic_algorithm(instance, n_population, n_iterations, min_mutation_rate=0, max_mutation_rate=1):
    population = generate_population(n_population, instance['n'])
    scores = [fitness(solution, instance) for solution in population]

    linucb = LinUCB(alpha=1, lambda_=1, min_value=min_mutation_rate, max_value=max_mutation_rate)
    mutation_rate = np.random.uniform(min_mutation_rate, max_mutation_rate)

    for i in range(n_iterations):
        
        print("Iteración ", i)
        
        new_population = []
        for _ in range(n_population):
            parent1 = selection(population, scores)
            parent2 = selection(population, scores)
            child = crossover(parent1, parent2)
            if np.random.rand() < mutation_rate:
                child = mutation(child)
            new_population.append(child)
        population, scores = replace_with_new_population(population, new_population, scores)
        
        min_cost = min(scores)
        print("Menor Costo:", min_cost)

        # Usamos el negativo del menor costo como la recompensa
        reward = -min_cost

        # Actualizamos LinUCB con la recompensa y el valor de mutation_rate utilizado
        linucb.update(reward, mutation_rate)

        # Obtenemos la nueva tasa de mutación de LinUCB
        mutation_rate = linucb.predict()
        
        print("Radio de mutacion:", mutation_rate)
        
    
    best_solution = population[np.argmin(scores)]
    best_score = min(scores)
    
    return best_solution, best_score


In [8]:
n_population = 20
n_iterations = 1500

In [14]:
results_ga = genetic_algorithm(instance, n_population, n_iterations, min_mutation_rate=0, max_mutation_rate=1)

Iteración  0
Menor Costo: 325936
Radio de mutacion: -126473.02040958898
Iteración  1
Menor Costo: 320354
Radio de mutacion: 2.5329811245569194
Iteración  2
Menor Costo: 319734
Radio de mutacion: 2.532930491551323
Iteración  3
Menor Costo: 315608
Radio de mutacion: 2.532880512924191
Iteración  4
Menor Costo: 315608
Radio de mutacion: 2.5328305352832747
Iteración  5
Menor Costo: 311532
Radio de mutacion: 2.532781204051688
Iteración  6
Menor Costo: 305800
Radio de mutacion: 2.532732781409366
Iteración  7
Menor Costo: 300490
Radio de mutacion: 2.5326852004839737
Iteración  8
Menor Costo: 294834
Radio de mutacion: 2.5326385160128093
Iteración  9
Menor Costo: 294834
Radio de mutacion: 2.5325918324022263
Iteración  10
Menor Costo: 294834
Radio de mutacion: 2.532545149652209
Iteración  11
Menor Costo: 294438
Radio de mutacion: 2.532498530461162
Iteración  12
Menor Costo: 294438
Radio de mutacion: 2.5324519121283386
Iteración  13
Menor Costo: 292382
Radio de mutacion: 2.532405620166871
Iteració

Menor Costo: 261246
Radio de mutacion: 2.526252890669025
Iteración  157
Menor Costo: 261246
Radio de mutacion: 2.5262116295241466
Iteración  158
Menor Costo: 261246
Radio de mutacion: 2.5261703690532333
Iteración  159
Menor Costo: 261246
Radio de mutacion: 2.5261291092562743
Iteración  160
Menor Costo: 261246
Radio de mutacion: 2.526087850133259
Iteración  161
Menor Costo: 261044
Radio de mutacion: 2.526046623585154
Iteración  162
Menor Costo: 261026
Radio de mutacion: 2.5260054005525445
Iteración  163
Menor Costo: 261026
Radio de mutacion: 2.5259641781927105
Iteración  164
Menor Costo: 261026
Radio de mutacion: 2.525922956505642
Iteración  165
Menor Costo: 261026
Radio de mutacion: 2.525881735491326
Iteración  166
Menor Costo: 261026
Radio de mutacion: 2.525840515149754
Iteración  167
Menor Costo: 261026
Radio de mutacion: 2.525799295480913
Iteración  168
Menor Costo: 261026
Radio de mutacion: 2.5257580764847933
Iteración  169
Menor Costo: 261026
Radio de mutacion: 2.5257168581613842


Menor Costo: 255276
Radio de mutacion: 2.519907490872992
Iteración  313
Menor Costo: 255276
Radio de mutacion: 2.519867273884066
Iteración  314
Menor Costo: 255276
Radio de mutacion: 2.5198270575370394
Iteración  315
Menor Costo: 255276
Radio de mutacion: 2.519786841831902
Iteración  316
Menor Costo: 255276
Radio de mutacion: 2.519746626768643
Iteración  317
Menor Costo: 255276
Radio de mutacion: 2.5197064123472526
Iteración  318
Menor Costo: 255276
Radio de mutacion: 2.5196661985677204
Iteración  319
Menor Costo: 255276
Radio de mutacion: 2.5196259854300367
Iteración  320
Menor Costo: 255276
Radio de mutacion: 2.51958577293419
Iteración  321
Menor Costo: 255276
Radio de mutacion: 2.519545561080172
Iteración  322
Menor Costo: 255276
Radio de mutacion: 2.5195053498679707
Iteración  323
Menor Costo: 255276
Radio de mutacion: 2.519465139297577
Iteración  324
Menor Costo: 255276
Radio de mutacion: 2.5194249293689794
Iteración  325
Menor Costo: 255276
Radio de mutacion: 2.5193847200821686
I

Menor Costo: 254254
Radio de mutacion: 2.5138946940350078
Iteración  463
Menor Costo: 254254
Radio de mutacion: 2.5138547336366286
Iteración  464
Menor Costo: 254254
Radio de mutacion: 2.513814773873501
Iteración  465
Menor Costo: 254254
Radio de mutacion: 2.513774814745613
Iteración  466
Menor Costo: 254254
Radio de mutacion: 2.5137348562529547
Iteración  467
Menor Costo: 254254
Radio de mutacion: 2.513694898395517
Iteración  468
Menor Costo: 254254
Radio de mutacion: 2.5136549411732894
Iteración  469
Menor Costo: 254254
Radio de mutacion: 2.513614984586262
Iteración  470
Menor Costo: 254254
Radio de mutacion: 2.5135750286344236
Iteración  471
Menor Costo: 254254
Radio de mutacion: 2.5135350733177653
Iteración  472
Menor Costo: 254254
Radio de mutacion: 2.513495118636276
Iteración  473
Menor Costo: 254254
Radio de mutacion: 2.5134551645899466
Iteración  474
Menor Costo: 254254
Radio de mutacion: 2.513415211178766
Iteración  475
Menor Costo: 254254
Radio de mutacion: 2.513375258402726


Menor Costo: 253020
Radio de mutacion: 2.5074882784581267
Iteración  624
Menor Costo: 253020
Radio de mutacion: 2.5074486133478002
Iteración  625
Menor Costo: 253020
Radio de mutacion: 2.50740894886497
Iteración  626
Menor Costo: 253020
Radio de mutacion: 2.5073692850096245
Iteración  627
Menor Costo: 253020
Radio de mutacion: 2.507329621781756
Iteración  628
Menor Costo: 253020
Radio de mutacion: 2.5072899591813527
Iteración  629
Menor Costo: 252974
Radio de mutacion: 2.5072503044189243
Iteración  630
Menor Costo: 252974
Radio de mutacion: 2.5072106502837137
Iteración  631
Menor Costo: 252974
Radio de mutacion: 2.507170996775711
Iteración  632
Menor Costo: 252974
Radio de mutacion: 2.5071313438949074
Iteración  633
Menor Costo: 252974
Radio de mutacion: 2.5070916916412913
Iteración  634
Menor Costo: 252974
Radio de mutacion: 2.5070520400148535
Iteración  635
Menor Costo: 252974
Radio de mutacion: 2.507012389015584
Iteración  636
Menor Costo: 252974
Radio de mutacion: 2.506972738643473

Menor Costo: 251134
Radio de mutacion: 2.501532112939
Iteración  775
Menor Costo: 251134
Radio de mutacion: 2.5014928370066007
Iteración  776
Menor Costo: 251134
Radio de mutacion: 2.5014535616909095
Iteración  777
Menor Costo: 251134
Radio de mutacion: 2.501414286991915
Iteración  778
Menor Costo: 251134
Radio de mutacion: 2.50137501290961
Iteración  779
Menor Costo: 251134
Radio de mutacion: 2.5013357394439835
Iteración  780
Menor Costo: 251134
Radio de mutacion: 2.501296466595026
Iteración  781
Menor Costo: 251134
Radio de mutacion: 2.5012571943627275
Iteración  782
Menor Costo: 251134
Radio de mutacion: 2.501217922747079
Iteración  783
Menor Costo: 251134
Radio de mutacion: 2.501178651748071
Iteración  784
Menor Costo: 251134
Radio de mutacion: 2.501139381365693
Iteración  785
Menor Costo: 251134
Radio de mutacion: 2.5011001115999347
Iteración  786
Menor Costo: 251134
Radio de mutacion: 2.5010608424507876
Iteración  787
Menor Costo: 251134
Radio de mutacion: 2.501021573918242
Itera

Menor Costo: 249412
Radio de mutacion: 2.4956645500764134
Iteración  925
Menor Costo: 249412
Radio de mutacion: 2.49562563494865
Iteración  926
Menor Costo: 249412
Radio de mutacion: 2.495586720427739
Iteración  927
Menor Costo: 249412
Radio de mutacion: 2.4955478065136716
Iteración  928
Menor Costo: 249412
Radio de mutacion: 2.495508893206438
Iteración  929
Menor Costo: 249400
Radio de mutacion: 2.4954699823781943
Iteración  930
Menor Costo: 249400
Radio de mutacion: 2.4954310721567072
Iteración  931
Menor Costo: 249286
Radio de mutacion: 2.4953921803269927
Iteración  932
Menor Costo: 249286
Radio de mutacion: 2.495353289103461
Iteración  933
Menor Costo: 249286
Radio de mutacion: 2.4953143984861024
Iteración  934
Menor Costo: 249286
Radio de mutacion: 2.4952755084749083
Iteración  935
Menor Costo: 249286
Radio de mutacion: 2.495236619069869
Iteración  936
Menor Costo: 249286
Radio de mutacion: 2.4951977302709745
Iteración  937
Menor Costo: 249286
Radio de mutacion: 2.4951588420782156

Menor Costo: 248032
Radio de mutacion: 2.489892903970827
Iteración  1074
Menor Costo: 248032
Radio de mutacion: 2.489854293662198
Iteración  1075
Menor Costo: 248032
Radio de mutacion: 2.4898156839523358
Iteración  1076
Menor Costo: 248032
Radio de mutacion: 2.4897770748412333
Iteración  1077
Menor Costo: 248032
Radio de mutacion: 2.489738466328879
Iteración  1078
Menor Costo: 247758
Radio de mutacion: 2.4896999010642142
Iteración  1079
Menor Costo: 247758
Radio de mutacion: 2.4896613363969573
Iteración  1080
Menor Costo: 247492
Radio de mutacion: 2.4896228137295413
Iteración  1081
Menor Costo: 247492
Radio de mutacion: 2.489584291658233
Iteración  1082
Menor Costo: 247492
Radio de mutacion: 2.4895457701830237
Iteración  1083
Menor Costo: 247492
Radio de mutacion: 2.4895072493039043
Iteración  1084
Menor Costo: 247492
Radio de mutacion: 2.489468729020865
Iteración  1085
Menor Costo: 247492
Radio de mutacion: 2.4894302093338974
Iteración  1086
Menor Costo: 247492
Radio de mutacion: 2.48

Menor Costo: 246504
Radio de mutacion: 2.4842160728384703
Iteración  1222
Menor Costo: 246504
Radio de mutacion: 2.4841777878763796
Iteración  1223
Menor Costo: 246504
Radio de mutacion: 2.4841395035043545
Iteración  1224
Menor Costo: 246504
Radio de mutacion: 2.484101219722384
Iteración  1225
Menor Costo: 246504
Radio de mutacion: 2.484062936530461
Iteración  1226
Menor Costo: 246504
Radio de mutacion: 2.484024653928575
Iteración  1227
Menor Costo: 246504
Radio de mutacion: 2.483986371916718
Iteración  1228
Menor Costo: 246504
Radio de mutacion: 2.48394809049488
Iteración  1229
Menor Costo: 246504
Radio de mutacion: 2.483909809663052
Iteración  1230
Menor Costo: 246504
Radio de mutacion: 2.4838715294212252
Iteración  1231
Menor Costo: 246504
Radio de mutacion: 2.4838332497693902
Iteración  1232
Menor Costo: 246504
Radio de mutacion: 2.483794970707539
Iteración  1233
Menor Costo: 246504
Radio de mutacion: 2.4837566922356604
Iteración  1234
Menor Costo: 246504
Radio de mutacion: 2.48371

Menor Costo: 246504
Radio de mutacion: 2.4785181146294613
Iteración  1371
Menor Costo: 246504
Radio de mutacion: 2.478479917486802
Iteración  1372
Menor Costo: 246504
Radio de mutacion: 2.4784417209328535
Iteración  1373
Menor Costo: 246504
Radio de mutacion: 2.4784035249676073
Iteración  1374
Menor Costo: 246504
Radio de mutacion: 2.4783653295910533
Iteración  1375
Menor Costo: 246504
Radio de mutacion: 2.478327134803183
Iteración  1376
Menor Costo: 246504
Radio de mutacion: 2.4782889406039867
Iteración  1377
Menor Costo: 246504
Radio de mutacion: 2.478250746993457
Iteración  1378
Menor Costo: 246504
Radio de mutacion: 2.478212553971584
Iteración  1379
Menor Costo: 246504
Radio de mutacion: 2.4781743615383576
Iteración  1380
Menor Costo: 246504
Radio de mutacion: 2.4781361696937703
Iteración  1381
Menor Costo: 246504
Radio de mutacion: 2.4780979784378117
Iteración  1382
Menor Costo: 246504
Radio de mutacion: 2.4780597877704733
Iteración  1383
Menor Costo: 246504
Radio de mutacion: 2.4